In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import os

import pandas as pd
import re
 
driver = webdriver.Firefox()

In [26]:
# Extract all categories links and names
def Categories_To_CSV():
    driver.get("https://www.midrag.co.il/Content/MainFeedbacksIndex")

    categories = driver.find_elements(By.CLASS_NAME, "sec-feedback-index")
    category_names = []
    urls = []

    for cat in categories:
        curr = cat.find_element(By.TAG_NAME,"a")

        #if curr.text in hot_categories:
        category_names.append((curr.text))
        urls.append(curr.get_attribute('href'))
        

    df = pd.DataFrame({"Category Name": category_names, "URL": urls, "Total Reivews": -1, "Is Relavent": 0, "Scanned": 0})
    Save_CSV("all_categories.csv", df)

In [25]:
# Update total reviews in CSV and sort accordingly
def Update_Total_Reivews(df):
    for index, url  in enumerate(df["URL"]):
        driver.get(url)

        total_reviews_XPATH = '/html/body/div[1]/div[1]/div[3]/div[1]/div[4]/div/div[1]/h1'
        total_reviews_raw = driver.find_elements(By.XPATH, total_reviews_XPATH)
        total_reviews = 0
        total_reviews = int(''.join(re.findall(r'\b\d+\b', total_reviews_raw[0].text)))

        df.iloc[index,3] = total_reviews
    
    df.sort_values(by=['Is Relevant', 'Scanned'],inplace=True, ascending=[False, False])
    Save_CSV("all_categories.csv", df)

# Update_Total_Reivews(df)

In [24]:
# Extract All Sub-Categories to CSV
def Get_All_Subcateogires_By_URL(cat_url):
    driver.get(cat_url)
    # Click on "כל האותיות" on page
    driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/div[3]/div[1]/div[4]/div/div[2]/a").click()
    # get all letters divs
    all_letters_raw = driver.find_elements(By.CLASS_NAME,"sec-feedback-index")

    all_letters_URL = []
    for letter in all_letters_raw:
        all_letters_URL.append(letter.find_element(By.TAG_NAME,'a').get_attribute('href'))

    print(all_letters_URL)
    # Call letter extractor for each link

# Get_All_Subcateogires_By_URL("https://www.midrag.co.il/Content/SecFeedbacksIndex/1")

In [22]:
def Append_Or_Create_CSV_By_Name(CSV_name, df):
    output_path=f'data/{CSV_name}'
    df.to_csv(output_path, mode='a', index =False, header=not os.path.exists(output_path), encoding = 'utf-8-sig')

In [23]:
def Save_CSV(CSV_name, df):
    df.to_csv(f"data/{CSV_name}", index=False, encoding = 'utf-8-sig')

In [20]:
# Get all reviews link from a letter URL and save those to CSV file
def Extract_All_Sub_Categories_URLS_From_Letter_Page(letter_url):
    driver.get(letter_url)

    CSV_file_name = "all_subcategories_URLs.csv"
    category_name = driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/div[3]/div[1]/div[1]/ol/li[2]/a/span").text
    all_links_raw = driver.find_elements(By.CLASS_NAME,"lttr-feedback-index")

    all_sub_link = []
    all_sub_name = []
    all_sub_count = []

    for link in all_links_raw:
        a_tag = link.find_element(By.TAG_NAME,'a')

        all_sub_link.append(a_tag.get_attribute('href'))

        sub_name = re.sub(r'\([^)]*\)', '', a_tag.text)
        count = re.findall(r'\d+', a_tag.text.replace(sub_name, "")).pop()
        
        all_sub_name.append(sub_name)
        all_sub_count.append(count)
        
    df = pd.DataFrame({"Category": category_name, "Subcategory" : all_sub_name,"Number Of Reviews" : all_sub_count,"Link" : all_sub_link, "scanned" : 0})
    Append_Or_Create_CSV_By_Name(CSV_file_name, df)

# Extract_All_Sub_Categories_URLS_From_Letter_Page("https://www.midrag.co.il/Content/LttrInSec/1?lttr=%D7%94&lttrAsc=228")

In [27]:
def Loop_All_Letter_Page(base_url):
    driver.get(base_url)
    num_of_pages = 1

    # try click on "דפים נוספים" if available and extract the total number of pages to scan
    try:
        more_pages_button = driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/div[3]/div[1]/div[4]/div[1]/div[2]/div/ul[2]/li/a")
        more_pages_button.click()

        # get the count of all button elements in the wrapper
        all_pages_buttons_wrapper = driver.find_element(By.CLASS_NAME, "pagination")
        all_pages_buttons = all_pages_buttons_wrapper.find_elements(By.TAG_NAME, "li")
        num_of_pages = len(all_pages_buttons)

    except:
        pass

    for page in range(1, num_of_pages + 1):
        current_url = base_url + f"&page={page}" 
        Extract_All_Sub_Categories_URLS_From_Letter_Page(current_url)

Loop_All_Letter_Page("https://www.midrag.co.il/Content/LttrInSec/1?lttr=%D7%94&lttrAsc=228")

דפים נוספים


In [ ]:
# Main loop: read and update csv files
# def Scrap_from_categories_CSV():
#     df = pd.read_csv("data/all_categories.csv")
#     for cat in df
#     print()